In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [12]:
import csv
import os
import numpy as np
import pandas as pd

BASE_PATH = 'raw_csv'
MFCC_DIR_PATH = 'mfcc'

data = []

for mfcc in os.listdir(f'{BASE_PATH}/{MFCC_DIR_PATH}/'):
    df_mfcc=pd.read_csv(f'{BASE_PATH}/{MFCC_DIR_PATH}/{mfcc}')
    mfcc = mfcc.replace('.csv','')
    result = ''.join([i for i in mfcc if not i.isdigit()])
    label = result[5:].upper()
    df_mfcc['label'] = label
    data.append(df_mfcc.values.tolist()[0])
columns_name = df_mfcc.columns
print(columns_name)
print(data)
print(len(data))

Index(['energy_coef', 'ceps_coef2', 'ceps_coef3', 'ceps_coef4', 'ceps_coef5',
       'ceps_coef6', 'ceps_coef7', 'ceps_coef8', 'ceps_coef9', 'ceps_coef10',
       'ceps_coef11', 'ceps_coef12', 'ceps_coef13', 'delta_energy_coef',
       'delta_ceps_coef2', 'delta_ceps_coef3', 'delta_ceps_coef4',
       'delta_ceps_coef5', 'delta_ceps_coef6', 'delta_ceps_coef7',
       'delta_ceps_coef8', 'delta_ceps_coef9', 'delta_ceps_coef10',
       'delta_ceps_coef11', 'delta_ceps_coef12', 'delta_ceps_coef13',
       'delta_delta_energy_coef', 'delta_delta_ceps_coef2',
       'delta_delta_ceps_coef3', 'delta_delta_ceps_coef4',
       'delta_delta_ceps_coef5', 'delta_delta_ceps_coef6',
       'delta_delta_ceps_coef7', 'delta_delta_ceps_coef8',
       'delta_delta_ceps_coef9', 'delta_delta_ceps_coef10',
       'delta_delta_ceps_coef11', 'delta_delta_ceps_coef12',
       'delta_delta_ceps_coef13', 'label'],
      dtype='object')
[[16.54138906234703, 13.172041872646336, 17.991153217797105, -7.62744171984

In [13]:
with open('data_mfcc.csv', 'w', newline='') as mfccfile:
    csv_out=csv.writer(mfccfile)
    csv_out.writerow(columns_name)
    for row in data:
        csv_out.writerow(row)

In [20]:
import pandas as pd
df=pd.read_csv('data_mfcc.csv')
df.head()

,energy_coef,ceps_coef2,ceps_coef3,ceps_coef4,ceps_coef5,ceps_coef6,ceps_coef7,ceps_coef8,ceps_coef9,ceps_coef10,...,delta_delta_ceps_coef5,delta_delta_ceps_coef6,delta_delta_ceps_coef7,delta_delta_ceps_coef8,delta_delta_ceps_coef9,delta_delta_ceps_coef10,delta_delta_ceps_coef11,delta_delta_ceps_coef12,delta_delta_ceps_coef13,label
0,16.541389,13.172042,17.991153,-7.627442,-8.032665,19.762998,-4.253296,-13.573726,-41.141261,0.439353,...,0.390865,-0.172088,-0.174607,0.070891,0.452843,-1.105071,-0.293982,0.735852,-0.302143,VE
1,17.589407,18.840204,31.210471,-5.523303,-5.413885,-21.414480,-50.288156,-19.975082,31.135011,-27.673481,...,0.143681,0.024281,0.068485,-0.204050,-0.131278,0.054081,-0.112385,-0.247590,-0.043061,WO
2,17.919551,-1.775943,-4.987889,62.793743,1.925696,-35.690793,-20.614873,12.509370,-22.099182,12.596949,...,-0.020678,0.117013,0.062955,-0.096598,0.045781,0.069156,0.055272,-0.011649,-0.095599,I
3,16.837870,22.843539,8.368299,-5.550276,7.494482,-9.041968,-53.631863,-11.474753,0.494225,-20.563528,...,-0.023687,0.050267,0.124827,-0.001373,-0.190014,-0.024350,0.071718,0.056307,0.042157,WO
4,17.405980,-1.420781,-17.052142,59.057986,18.079349,59.597624,-36.856457,1.547511,-45.547640,16.802576,...,-0.370561,-0.349142,0.090947,0.128286,0.564285,-0.421928,0.619782,-0.449197,0.295639,I


In [4]:
df = df.dropna()
df.shape

(2400, 40)

In [5]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, cross_validate, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.ensemble import RandomForestClassifier

train = df.copy()
y = train.pop('label').values
X = train

kf = KFold(n_splits=10, shuffle=True)

@ignore_warnings(category=Warning)
def kf_pred(clf, X, y):
    accuracy = []
    precision = []
    recall = []
    i = 1
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        predict = cross_val_predict(clf, X_test, y_test, cv=10)
        #print('Iteration: ', i)
        #print('First 5 predicted: ', predict[:5])
        #print('First 5 real: ', y_test[:5])
        #print("Accuracy:", accuracy_score(y_test, predict, normalize=True))
        #print("Precision:", precision_score(y_test, predict, average='macro'))
        #print("Recall:", recall_score(y_test, predict, average='macro'))
        accuracy.append(accuracy_score(y_test, predict, normalize=True))
        precision.append(precision_score(y_test, predict, average='macro'))
        recall.append(recall_score(y_test, predict, average='macro'))
        i+=1
    return accuracy, precision, recall

In [6]:
vowels = ['A', 'I', 'U', 'E', 'O', 'EU']
vowel_data = df[df['label'].isin(vowels)]
y_vowel = vowel_data.pop('label').values
X_vowel = vowel_data
other_data = df[~df['label'].isin(vowels)]
y_other = other_data.pop('label').values
X_other = other_data

X_vowel

,energy_coef,ceps_coef2,ceps_coef3,ceps_coef4,ceps_coef5,ceps_coef6,ceps_coef7,ceps_coef8,ceps_coef9,ceps_coef10,...,delta_delta_ceps_coef4,delta_delta_ceps_coef5,delta_delta_ceps_coef6,delta_delta_ceps_coef7,delta_delta_ceps_coef8,delta_delta_ceps_coef9,delta_delta_ceps_coef10,delta_delta_ceps_coef11,delta_delta_ceps_coef12,delta_delta_ceps_coef13
2,17.919551,-1.775943,-4.987889,62.793743,1.925696,-35.690793,-20.614873,12.509370,-22.099182,12.596949,...,-0.088891,-0.020678,0.117013,0.062955,-0.096598,0.045781,0.069156,0.055272,-0.011649,-0.095599
4,17.405980,-1.420781,-17.052142,59.057986,18.079349,59.597624,-36.856457,1.547511,-45.547640,16.802576,...,0.065081,-0.370561,-0.349142,0.090947,0.128286,0.564285,-0.421928,0.619782,-0.449197,0.295639
9,14.902087,15.416801,0.434152,5.096780,3.480033,10.583830,-6.523984,-1.104637,-10.266595,-22.919026,...,-0.098039,-0.003546,-0.106635,0.092955,0.024651,0.029994,-0.012862,-0.098866,-0.027692,0.023046
10,17.253012,-15.670014,16.786858,48.407346,-24.622470,-12.528667,-13.747756,-22.749430,-29.469509,-14.608503,...,-0.166841,-0.002514,0.156692,0.130358,0.084880,0.068199,0.245015,0.205365,-0.015020,0.129517
13,16.121101,7.573440,-8.954514,18.910829,30.773467,21.960115,-22.545092,-36.617723,-6.026905,4.726953,...,-0.408892,-0.459361,-0.094665,0.435285,0.363619,-0.228727,-0.165013,0.077507,0.193141,0.086102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,17.140412,28.751530,15.432901,21.283986,-7.594644,-2.215246,-49.217079,-20.117869,14.935334,4.250760,...,0.003198,0.171560,-0.010861,0.240541,-0.007645,-0.109845,-0.107204,0.236309,0.044567,-0.039333
2385,17.979455,-5.242553,-8.148795,37.315857,-27.218498,-39.449142,-26.726179,2.309669,-2.254994,-10.188963,...,-0.190497,0.016982,0.236289,0.007495,0.172340,0.028597,0.212873,0.081018,-0.005847,0.044420
2389,16.613772,41.979057,-22.701342,7.922777,19.344647,48.888253,25.628233,-19.254023,-28.462371,-35.834980,...,-0.036963,0.097682,-0.067434,0.073149,-0.059967,-0.206054,0.183316,-0.034369,0.004197,-0.044943
2390,16.471354,39.059047,-21.227991,18.778252,10.597818,45.197497,32.440398,-15.446940,-25.213279,-36.974775,...,-0.323116,0.174105,-0.111330,-0.058332,0.090339,0.006304,0.137496,-0.197194,0.077890,0.102715


## Classification

### Decision Tree

In [9]:
tree_clf_kf_vowel = DecisionTreeClassifier(criterion='entropy', max_leaf_nodes=16)
tree_clf_kf_other = DecisionTreeClassifier(criterion='entropy', max_leaf_nodes=16)

print('Vowels Tree Classification')

accuracy, precision, recall = kf_pred(tree_clf_kf_vowel, X_vowel, y_vowel)
print('Accuracy mean: ', np.mean(accuracy))
print('Precision mean: ', np.mean(precision))
print('Recall mean: ', np.mean(recall))
print('\n')

print('Other Tree Classification')

accuracy, precision, recall = kf_pred(tree_clf_kf_other, X_other, y_other)
print('Accuracy mean: ', np.mean(accuracy))
print('Precision mean: ', np.mean(precision))
print('Recall mean: ', np.mean(recall))

Vowels Tree Classification
Accuracy mean:  0.4133333333333334
Precision mean:  0.39290497983592615
Recall mean:  0.3924849605731958


Other Tree Classification
Accuracy mean:  0.1922222222222222
Precision mean:  0.1489332908769931
Recall mean:  0.1712718454385121


In [10]:
rf_vowel = RandomForestClassifier()
rf_other = RandomForestClassifier()
print('Vowels RF Classification')

accuracy, precision, recall = kf_pred(rf_vowel, X_vowel, y_vowel)
print('Accuracy mean: ', np.mean(accuracy))
print('Precision mean: ', np.mean(precision))
print('Recall mean: ', np.mean(recall))
print('\n')

print('Other RF Classification')

accuracy, precision, recall = kf_pred(rf_other, X_other, y_other)
print('Accuracy mean: ', np.mean(accuracy))
print('Precision mean: ', np.mean(precision))
print('Recall mean: ', np.mean(recall))

Vowels RF Classification
Accuracy mean:  0.385
Precision mean:  0.360654017178785
Recall mean:  0.34926157176157174


Other RF Classification
Accuracy mean:  0.23055555555555557
Precision mean:  0.22387463756478207
Recall mean:  0.21184044350711018


### Neural Network

In [11]:
from sklearn.neural_network import MLPClassifier

nn_vowel = MLPClassifier(solver='adam', alpha=1e-4,
                        hidden_layer_sizes=(30, 30), random_state=42, batch_size=32, max_iter=1000)
nn_other = MLPClassifier(solver='adam', alpha=1e-4,
                        hidden_layer_sizes=(30, 30), random_state=42, batch_size=32, max_iter=1000)

print('Vowels Neural Network Classification')

accuracy, precision, recall = kf_pred(nn_vowel, X_vowel, y_vowel)
print('Accuracy mean: ', np.mean(accuracy))
print('Precision mean: ', np.mean(precision))
print('Recall mean: ', np.mean(recall))
print('\n')

print('Other Neural Network Classification')

accuracy, precision, recall = kf_pred(nn_other, X_other, y_other)
print('Accuracy mean: ', np.mean(accuracy))
print('Precision mean: ', np.mean(precision))
print('Recall mean: ', np.mean(recall))

Vowels Neural Network Classification
Accuracy mean:  0.5933333333333333
Precision mean:  0.5865931753431753
Recall mean:  0.5761558811558812


Other Neural Network Classification
Accuracy mean:  0.33166666666666667
Precision mean:  0.31566434080772593
Recall mean:  0.31221661787644756
